In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
# N_out = (N_in + 2p - k / s) -1

In [3]:
class CNN(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    super(CNN, self).__init__()

    self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=(3, 3), stride=1, padding=1)
    self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), stride=1, padding=1)

    self.fc = nn.Linear(32*7*7, num_classes) 

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc(x)
    return x

In [4]:
model = CNN()

In [5]:
x = torch.randn(64, 1, 28, 28)

In [6]:
print(model(x).shape)

torch.Size([64, 10])


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [8]:
model.to(device)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

In [9]:
# Hyperparameters
in_channels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 30

In [10]:
train_dataset = datasets.MNIST(root="dataset/", train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size = batch_size)

test_dataset = datasets.MNIST(root="dataset/", train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_dataset, shuffle=True, batch_size = batch_size)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [17]:
model = CNN(in_channels=in_channels, num_classes=num_classes)

In [18]:
model.to(device)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

In [19]:
criterion = nn.CrossEntropyLoss()

In [20]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)

    # Forward
    scores = model(data)
    loss = criterion(scores, targets)

    # Backward
    optimizer.zero_grad()
    loss.backward()

    # Update Gradients
    optimizer.step()
  print(f"[*] Epoch {epoch+1} loss={loss}")

[*] Epoch 1 loss=0.008229234255850315
[*] Epoch 2 loss=0.06605736166238785
[*] Epoch 3 loss=0.03440805524587631
[*] Epoch 4 loss=0.0046742577105760574
[*] Epoch 5 loss=0.022462904453277588
[*] Epoch 6 loss=0.00031134349410422146
[*] Epoch 7 loss=0.004019421525299549
[*] Epoch 8 loss=0.015444286167621613
[*] Epoch 9 loss=0.005684663075953722
[*] Epoch 10 loss=0.020567134022712708
[*] Epoch 11 loss=0.010508626699447632
[*] Epoch 12 loss=1.94205749721732e-05
[*] Epoch 13 loss=0.00026937597431242466
[*] Epoch 14 loss=0.15850795805454254
[*] Epoch 15 loss=0.00013758649583905935
[*] Epoch 16 loss=0.0002106571919284761
[*] Epoch 17 loss=0.04412184655666351
[*] Epoch 18 loss=0.001434846199117601
[*] Epoch 19 loss=0.001233284710906446
[*] Epoch 20 loss=0.003929889760911465
[*] Epoch 21 loss=0.00025169242871925235
[*] Epoch 22 loss=6.580154149560258e-05
[*] Epoch 23 loss=1.4108228242548648e-05
[*] Epoch 24 loss=9.465132279729005e-06
[*] Epoch 25 loss=3.986056071880739e-07
[*] Epoch 26 loss=8.507

In [22]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training dataset")
  else:
    print("Checking accuracy on testing dataset")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device)
      y = y.to(device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)
    
    acc = float(num_correct)/float(num_samples)*100
    print(f"{num_correct}/{num_samples} with accuracy {acc:.2f}.")
  
  model.train()
  return acc

In [23]:
check_accuracy(train_loader, model)

Checking accuracy on training dataset
59929/60000 with accuracy 99.88.


99.88166666666667

In [24]:
check_accuracy(test_loader, model)

Checking accuracy on testing dataset
9882/10000 with accuracy 98.82.


98.82